In [ ]:
import torch
print(torch.cuda.current_device())

In [12]:
!python --version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Python 3.10.4


In [4]:
!nvidia-smi

Fri May 12 01:34:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:17:00.0 Off |                    0 |
| N/A   32C    P0    33W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:CA:00.0 Off |                   19 |
| N/A   

In [ ]:
# !conda install pytorch torchvision -c pytorch --yes
# !pip install torch torchvision

In [ ]:
# !python -m pip install  torch transformers
# !pip install torch torchvision transformers
# !conda install pytorch torchvision  -c pytorch


In [ ]:
# !which pip
# !conda install -c conda-forge python=3.7 python pip3
# !conda install -c conda-forge transformers
# !conda install pytorch torchvision --yes

# !pip uninstall transformers --yes
# #!pip uninstall torch --yes
# !pip install transformers
# !pip install torch --yes
# # !pip install transformers --yes
# # !pip install cython --yes


# from datasets import load_dataset
# datase = load_dataset("json", data_files="te.json")


In [1]:
!pip install torch
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install rouge_score
!pip install bert_score
!pip install bleurt

ERROR: Could not find a version that satisfies the requirement bleurt (from versions: none)
ERROR: No matching distribution found for bleurt


In [55]:

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from bert_score import score
import datasets
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from math import exp

import json
import math
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import argparse
import logging
from torch.utils.data import IterableDataset
import gzip
import json
import sys
from datetime import datetime
import random
from shutil import copyfile
import os
import re
# from huggingface_hub import Repository
# repo = Repository("gpt2-reddit", clone_from="skunusot/gpt2-reddit")
# repo.git_pull()

output_dir_main = "./reddit-model"

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", default="gpt2")
parser.add_argument("--train_files", required=False, nargs='+', default=['./combinedcleaned-full.jsonl.gz'])
parser.add_argument("--name", required=False, default="reddit")
parser.add_argument("--train_size", default=320000, type=int)
parser.add_argument("--eval_size", default=50, type=int)
parser.add_argument("--test_size", default=80000, type=int)
args = parser.parse_args(args=[])

In [56]:

def get_example(s):
        raw_example = json.loads(s)
        return [(clean_title(raw_example['title'])), clean_title(raw_example['body'])]
    
def clean_title(text):
    text = text.replace("&amp;", "&").strip()
    if text.startswith("["):
        text = re.sub("^\[[a-zA-Z0-9]+\]", "", text).strip()

    if text.endswith("]"):
        text = re.sub("\[[a-zA-Z0-9\.]+\]$", "", text).strip()

    if text.startswith("/r"):
        text = re.sub("^/[a-zA-Z0-9/]+[;,: \-]+", "", text).strip()

    return text
    
class RedditTitleDataset(IterableDataset):
    def __init__(self, filepath):
        self.filepath = filepath
        self.examples = []
    
    def __iter__(self):
        with gzip.open(self.filepath, 'rt', encoding='utf-8') as fIn:
            for line in fIn:
                example = get_example(json.loads(line))
                if example is not None and example[0] != '' and example[1] != '':
                    self.examples.append(example)
                    yield example
        
    def get_total(self):
        total = 0
        with gzip.open(self.filepath, 'rt', encoding='utf-8') as fIn:
            for line in fIn:
                example = get_example(json.loads(line))
                if example[0] != '' and example[1] != '':
                    total += 1
        
        return total
        
    
    def __len__(self):
        return len(self.examples)

device = torch.device("cuda")

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(args.model_name)

tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': tokenizer.bos_token, 'eos_token': tokenizer.eos_token})


# Define data collator
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )
def data_collator(examples):
    # print(examples)
    if examples is None: 
        return None
    targets = [row[0] for row in examples]
    inputs = [row[1] for row in examples]
    # print("targets: ")
    # print(targets)
    # print("inputs: ")
    # print(inputs)
    model_inputs = tokenizer(targets[0], inputs[0], padding=True, truncation=True, return_tensors='pt', pad_to_multiple_of=8)

    if not model_inputs.data["input_ids"] is None or len(model_inputs.data["input_ids"]) == 0:
        # print("model_inputs")
        # print(model_inputs.data["input_ids"])
        model_inputs["labels"] = torch.tensor(model_inputs.data["input_ids"]).to(device)
    return model_inputs

    

import gc

# Create dataset and dataloader
reddit_dataset = RedditTitleDataset(args.train_files[0])
reddit_dataset_iter = iter(reddit_dataset)

train_dataset = [next(reddit_dataset_iter) for _ in range(470000 - args.eval_size - args.test_size-100)] # 470000 #5200087
eval_dataset = [next(reddit_dataset_iter) for _ in range(args.eval_size)]
test_dataset = [next(reddit_dataset_iter) for _ in range(args.test_size)]
del reddit_dataset
del reddit_dataset_iter

gc.collect()



print("Target:", eval_dataset[5][0])
print("Input:", eval_dataset[5][1])


open ./combinedcleaned-full.jsonl.gz
Target: Its time for you to sing
Input: Row, row, row your boat gently down the stream! Thats all I know.


In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir_main,
    evaluation_strategy="steps",
    eval_steps=250,
#     save_total_limit=2,
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    hub_strategy="every_save",
    save_strategy="no",
    push_to_hub=False,
    hub_model_id="skunusot/gpt2-reddit",
    hub_private_repo=True,
    hub_token="whf_LbwUQBNXqnUndGiCJePZLvNzcVRQCOXtSI",
    dataloader_pin_memory=False
)


model = GPT2LMHeadModel.from_pretrained(args.model_name).to(device)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()
# # trainer.push_to_hub()

trainer.save_model(output_dir_main)
tokenizer.save_pretrained(output_dir_main) 
    


